In [4]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Small
from google.cloud import storage

In [1]:
MODEL_FILENAME = 'mobilenetv3small.keras'
GCS_BUCKET_NAME = 'tf_models_cv'
GCS_MODEL_PATH = 'v3_0/v3_0.keras'

In [5]:
model = MobileNetV3Small(weights='imagenet')

# Save model locally in SavedModel format
model.save(MODEL_FILENAME)


/opt/homebrew/Caskroom/miniconda/base/envs/creature-vis-0.1/lib/python3.11/site-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


In [6]:
# Function to upload a single file to GCS
def upload_file_to_gcs(local_file, bucket_name, destination_blob_name):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(local_file)
    print(f"Uploaded {local_file} to gs://{bucket_name}/{destination_blob_name}")

In [7]:
upload_file_to_gcs(MODEL_FILENAME, GCS_BUCKET_NAME, GCS_MODEL_PATH)

Uploaded mobilenetv3small.keras to gs://tf_models_cv/v3_0/v3_0.keras


In [ ]:
from io import StringIO

# === CONFIG ===
prev_version = "v3_0"  # Replace with your model version
bucket = "tf_models_cv"  # Your model bucket
artifact_path = f"gs://{bucket}/{prev_version}/{prev_version}_summary.txt"

# === LOAD MODEL FROM GCS ===
model_path = f"gs://{bucket}/{prev_version}/{prev_version}.keras"
model = tf.keras.models.load_model(model_path)
print(f"Loaded model from: {model_path}")

# === CAPTURE SUMMARY ===
def get_model_summary(model):
    stream = StringIO()
    model.summary(print_fn=lambda x: stream.write(x + "\n"))
    summary_str = stream.getvalue()
    stream.close()
    return summary_str

summary_text = get_model_summary(model)

# === WRITE TO GCS ===
with tf.io.gfile.GFile(artifact_path, "w") as f:
    f.write(summary_text)

print(f"Model summary written to: {artifact_path}")


Loaded model from: gs://tf_models_cv/v3_0/v3_0.keras


Model summary written to: gs://tf_models_cv/v3_0/v3_0_summary.txt
